# 1. Environment & Multi-GPU Strategy

In [1]:
import os, cv2, warnings
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from skimage.exposure import match_histograms

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")
strategy = tf.distribute.MirroredStrategy()

2026-01-22 20:08:03.028047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769112483.054134    3827 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769112483.061758    3827 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769112483.081957    3827 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769112483.081987    3827 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769112483.081990    3827 computation_placer.cc:177] computation placer alr

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


I0000 00:00:1769112487.036337    3827 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1769112487.040293    3827 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


# 2. Dataset Paths

In [2]:
BR35H_YES = "/kaggle/input/brain-tumor-detection/yes"
BR35H_NO  = "/kaggle/input/brain-tumor-detection/no"
NAV_YES = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes"
NAV_NO  = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/no"
MOS_YES = "/kaggle/input/brain-tumor-mri-yes-or-no/Brain (y-n)/Training/yes"
MOS_NO  = "/kaggle/input/brain-tumor-mri-yes-or-no/Brain (y-n)/Training/no"

# 3. Image Processing & Loading

In [3]:
def crop_brain(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    if len(cnts) == 0:
        return cv2.resize(image, (224, 224))
    c = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    return cv2.resize(image[y:y+h, x:x+w], (224, 224))

def load_dataset(yes_dir, no_dir, name):
    imgs, lbls = [], []
    for folder, label in [(yes_dir, 1), (no_dir, 0)]:
        files = os.listdir(folder)
        for fname in files:
            img = cv2.imread(os.path.join(folder, fname))
            if img is not None:
                imgs.append(crop_brain(img))
                lbls.append(label)
    X, y = np.array(imgs, dtype=np.uint8), np.array(lbls, dtype=np.int32)
    pos = np.sum(y)
    print(f"{name} Load Complete | Total: {len(y)} | Yes: {pos} | No: {len(y)-pos}")
    return X, y

X_train_raw, y_train = load_dataset(BR35H_YES, BR35H_NO, "TRAIN")
X_val_raw, y_val = load_dataset(NAV_YES, NAV_NO, "VAL")
X_test_raw, y_test = load_dataset(MOS_YES, MOS_NO, "TEST")

TRAIN Load Complete | Total: 3000 | Yes: 1500 | No: 1500
VAL Load Complete | Total: 253 | Yes: 155 | No: 98
TEST Load Complete | Total: 5450 | Yes: 2725 | No: 2725


# 4. Heavy Augmentation & Global Preprocessing

In [4]:
_rng = np.random.default_rng(42)
_ref_pool = X_train_raw[_rng.choice(len(X_train_raw), size=min(256, len(X_train_raw)), replace=False)]

def _safe_preprocess(x):
    return np.clip(np.nan_to_num(x, nan=0.0, posinf=255.0, neginf=0.0), 0.0, 255.0).astype(np.float32)

def preprocess_train(x):
    x_rgb = _safe_preprocess(x[..., ::-1])
    if _rng.random() < 0.30:
        ref_rgb = _safe_preprocess(_ref_pool[_rng.integers(0, len(_ref_pool))][..., ::-1])
        x_rgb = _safe_preprocess(match_histograms(x_rgb, ref_rgb, channel_axis=-1))
    return resnet_preprocess(x_rgb)

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_train,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    shear_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.7, 1.3),
    fill_mode="nearest"
)

train_flow = datagen.flow(X_train_raw, y_train, batch_size=100, shuffle=True, seed=42)
X_val_p = resnet_preprocess(_safe_preprocess(X_val_raw[..., ::-1]))
X_test_p = resnet_preprocess(_safe_preprocess(X_test_raw[..., ::-1]))

# 5. Model Building (Conv4 Unfrozen & Max Pooling)

In [5]:
with strategy.scope():
    base = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    
    base.trainable = True
    trainable_marker = False
    for layer in base.layers:
        if "conv4_" in layer.name:
            trainable_marker = True
        if not trainable_marker:
            layer.trainable = False
            
    x = layers.GlobalMaxPooling2D()(base.output)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    out = layers.Dense(1, activation="sigmoid", dtype="float32")(x)
    
    model = models.Model(inputs=base.input, outputs=out)
    model.compile(
        optimizer=Adam(learning_rate=2e-4),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.01),
        metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
    )

# 6. Training Execution

In [6]:
spe = len(X_train_raw) // 100

callbacks = [
    EarlyStopping(monitor="val_auc", mode="max", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_auc", mode="max", patience=2, factor=0.2, min_lr=1e-7)
]

model.fit(
    train_flow,
    steps_per_epoch=spe,
    validation_data=(X_val_p, y_val),
    epochs=15,
    callbacks=callbacks,
    verbose=1
)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

I0000 00:00:1769112557.076675    3885 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1769112557.118477    3886 cuda_dnn.cc:529] Loaded cuDNN version 91002


30/30 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.7603 - auc: 0.8282 - loss: 0.5782 - val_accuracy: 0.9526 - val_auc: 0.9856 - val_loss: 0.2703 - learning_rate: 2.0000e-04
Epoch 2/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.9317 - auc: 0.9808 - loss: 0.2079 - val_accuracy: 0.9723 - val_auc: 0.9982 - val_loss: 0.1592 - learning_rate: 2.0000e-04
Epoch 3/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9631 - auc: 0.9941 - loss: 0.1302 - val_accuracy: 0.9763 - val_auc: 0.9932 - val_loss: 0.1578 - learning_rate: 2.0000e-04
Epoch 4/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9767 - auc: 0.9968 - loss: 0.1010 - val_accuracy: 0.9881 - val_auc: 0.9911 - val_loss: 0.1186 - learning_rate: 2.0000e-04
Epoch 5/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9837 - auc: 0.9982 - loss: 0.0838 - val_accuracy: 0.9881 - val_auc: 0.9963 - val_loss: 0.1038 - learning_rate: 4.0000e-05
Epoch 6/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9832 - auc: 0.996

# 7. Final Evaluation & Saving

In [7]:
results = model.evaluate(X_test_p, y_test, verbose=1)
print(f"Test Stats | Acc: {results[1]:.4f} | AUC: {results[2]:.4f}")

model.save("optimized_brain_tumor_resnet50.keras")

171/171 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step - accuracy: 0.8705 - auc: 0.4351 - loss: 0.4998
Test Stats | Acc: 0.8007 | AUC: 0.8599
